#**t5-base-korean-paraphrase model Fine-tunning: 일상대화 성능 upgrade**
Chang W Lee가 만듦.
huggingface: https://huggingface.co/lcw99/t5-base-korean-paraphrase

# **Install and Import libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00


In [ ]:
#importing libraries
from torch.utils.data import DataLoader, Dataset, RandomSampler, BatchSampler
from torch.nn.functional import pad
from torch.nn.utils.rnn import pad_sequence
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW, get_linear_schedule_with_warmup, AutoModelForCausalLM
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP프로젝트/데이터/train_data/train_data.csv")
df.head()

,Unnamed: 0,texts,pairs
0,0,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,9,사람이 고양이를 천장에 던지고 있다.,사람이 고양이를 천장에 던진다.
2,11,한 여성이 아기를 안아서 캥거루를 안는다.,한 여성이 아기를 안아서 팔에 캥거루를 안는다.
3,13,사람이 종이 한 장을 접고 있다.,누군가가 종이를 접고 있다.
4,16,북극곰이 눈 위에서 미끄러지고 있다.,북극곰이 눈 위로 미끄러져 가고 있다.


In [ ]:
from tqdm import tqdm

class StsDataset(Dataset):
    def __init__(self, tokenizer, df):
        self.input_ids = []
        self.target_ids = []
        self.attention_masks = []
        for _, row in tqdm(df.iterrows()):
            inputencoding = tokenizer("Paraphrase: "+row.values[1]+"</s>", padding=True,truncation = True, return_tensors = 'pt')
            input = inputencoding.input_ids
            input = torch.squeeze(input)
            attention_mask = inputencoding.attention_mask
            attention_mask = torch.squeeze(attention_mask)
            target = tokenizer(row.values[2]+"</s>", padding=True,truncation = True, return_tensors = 'pt').input_ids
            target = torch.squeeze(target)
            max_len = max(input.size()[-1], attention_mask.size()[-1], target.size()[-1])
            if max_len - input.size()[-1] > 0:
                input = torch.cat((input, torch.zeros(1)*(max_len - input.size()[-1])), dim = -1)
                attention_mask = torch.cat((attention_mask, torch.zeros(1)*(max_len - input.size()[-1])), dim = -1)
            target = torch.cat((target, torch.zeros(1)*(max_len - target.size()[-1])), dim = -1) if max_len - target.size()[-1] > 0 else target
            self.input_ids.append(input)
            self.target_ids.append(target)
            self.attention_masks.append(attention_mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        self.input_ids[idx] = self.input_ids[idx].type(torch.LongTensor)
        self.attention_masks[idx] = self.attention_masks[idx].type(torch.LongTensor)
        self.target_ids[idx] = self.target_ids[idx].type(torch.LongTensor)
        return {"input_ids": self.input_ids[idx], "attention_masks": self.attention_masks[idx], "target_ids": self.target_ids[idx]}

In [ ]:
model_path = "lcw99/t5-base-korean-paraphrase"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
def collate_fn(batchDummy):
  x = [torch.LongTensor(batch["input_ids"]) for batch in batchDummy], [torch.LongTensor(batch["attention_masks"]) for batch in batchDummy], [torch.LongTensor(batch["target_ids"]) for batch in batchDummy]
  # batch단위로 데이터가 넘어올 때 아래 pad_sequence를 통해 알아서 padding을 해준다
  input = torch.nn.utils.rnn.pad_sequence(x[0], batch_first = True)
  attention_masks = torch.nn.utils.rnn.pad_sequence(x[1], batch_first = True)
  target_ids = torch.nn.utils.rnn.pad_sequence(x[2], batch_first = True)
  return {"input_ids": input, "attention_masks": attention_masks, "target_ids": target_ids}

#배치사이즈, 사용할 데이터프레임
Batch_Size = 8
pairs = df

# 추가 학습을 위한 모델 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
5


num_epochs = 2  # 추가 학습할 에폭 수
dataset = StsDataset(tokenizer, pairs)

test_val_split_idx = int(pairs.values.shape[0]*0.2)

train_point_sampler = RandomSampler([i for i in range(test_val_split_idx, pairs.values.shape[0])])
train_sampler = BatchSampler(train_point_sampler, Batch_Size, False)
trainDataLoader = DataLoader(dataset, collate_fn = collate_fn, batch_sampler = train_sampler)

# 학습 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps= ((pairs.values.shape[0]-test_val_split_idx)//Batch_Size + 1) * num_epochs)

0it [00:00, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
225376it [02:44, 1369.42it/s]


In [ ]:
# 추가 학습 실행
for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_acc = 0.0

    for i, data in enumerate(trainDataLoader):
        optimizer.zero_grad()
        input_ids, attention_masks, target_ids = (data.values())
        outputs = model(input_ids= input_ids.to(device),attention_mask = attention_masks.to(device),labels=target_ids.to(device))
        loss = outputs.loss
        logits = outputs.logits
        preds = logits.argmax(dim=-1)

        accuracy = 0
        for j in range(len(input_ids)):
            acc_score = accuracy_score(target_ids[j].cpu(), preds[j].cpu())
            accuracy += acc_score
        accuracy /= len(input_ids)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += accuracy

        # 학습 상황 출력
        print(f"Epoch [{epoch+1}/{num_epochs}] | Step [{i+1}/{(pairs.values.shape[0] - test_val_split_idx)//Batch_Size+1}] | Loss: {loss.item():.4f} | Accuracy: {accuracy:.4f}")

    scheduler.step()
    epoch_loss /= (pairs.values.shape[0] - test_val_split_idx)//Batch_Size + 1
    epoch_acc /= (pairs.values.shape[0] - test_val_split_idx)//Batch_Size + 1
    print(f"Epoch {epoch+1} Loss: {epoch_loss} | Accuracy: {epoch_acc}")

# 추가 학습된 모델 저장
model.save_pretrained(f"/content/drive/MyDrive/T5model_{pairs.values.shape[0]}data")
tokenizer.save_pretrained(f"/content/drive/MyDrive/T5model_{pairs.values.shape[0]}data")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch [2/2] | Step [17540/22538] | Loss: 0.3341 | Accuracy: 0.9014
Epoch [2/2] | Step [17541/22538] | Loss: 0.2909 | Accuracy: 0.9313
Epoch [2/2] | Step [17542/22538] | Loss: 0.3684 | Accuracy: 0.8963
Epoch [2/2] | Step [17543/22538] | Loss: 0.3372 | Accuracy: 0.8934
Epoch [2/2] | Step [17544/22538] | Loss: 0.2563 | Accuracy: 0.9256
Epoch [2/2] | Step [17545/22538] | Loss: 0.2787 | Accuracy: 0.9206
Epoch [2/2] | Step [17546/22538] | Loss: 0.1939 | Accuracy: 0.9435
Epoch [2/2] | Step [17547/22538] | Loss: 0.2652 | Accuracy: 0.9250
Epoch [2/2] | Step [17548/22538] | Loss: 0.3706 | Accuracy: 0.8750
Epoch [2/2] | Step [17549/22538] | Loss: 0.3915 | Accuracy: 0.8883
Epoch [2/2] | Step [17550/22538] | Loss: 0.1843 | Accuracy: 0.9367
Epoch [2/2] | Step [17551/22538] | Loss: 0.2153 | Accuracy: 0.9304
Epoch [2/2] | Step [17552/22538] | Loss: 0.2703 | Accuracy: 0.9180
Epoch [2/2] | Step [17553/22538] | Loss: 0.3863 | Accuracy: 0.9047
Epoch [2/2] | Step [17554/

('/content/drive/MyDrive/T5model_225376data/tokenizer_config.json',
 '/content/drive/MyDrive/T5model_225376data/special_tokens_map.json',
 '/content/drive/MyDrive/T5model_225376data/tokenizer.json')

In [ ]:
def getParaphrased(s):
    encoded = tokenizer("Paraphrase: "+s, padding=True, truncation = True, return_tensors = 'pt')
    output = model.generate(
        input_ids=encoded["input_ids"].to(device), attention_mask=encoded["attention_mask"].to(device), # input_ids = 인코딩된 입력 문장의 인덱스 텐서 / attention_mask = 인코딩된 입력 문장의 어텐션 마스크 텐서(0=패딩토큰, 1=실제토큰)
        max_length=256, # 생성되는 문장의 최대 출력 길이
        do_sample=True, # 샘플링 여부 파라미터(True=다양한 토큰을 샘플링하여 출력 생성 / False=모델이 가장 확률적으로 높은 토큰을 선택하여 출력 생성)
        top_k=50, # 상위 k개의 확률 분포에서만 토큰을 샘플링하는데 사용되는 파라미터. 값을 높이면 생성 다양성 증가. 낮추면 생성 정확성 증가.
        top_p=0.9, # 누적 확률 분포의 상위 p%에 해당하는 토큰만을 고려하여 샘플링하는데 사용되는 파라미터. 값을 높이면 생성 정확성이 증가, 낮추면 생성 다양성이 증가.
        early_stopping=False, # 조기 중단 여부 결정 파라미터(True=모델이 '<eos>' 토큰을 만났을 때 생성 중단 / False=max_length까지 계속해서 토큰 생성)
        num_return_sequences=5 # 생성할 문장 개수 지정 파라미터. 출력 시퀀스 1개로 설정
    )
    result = []
    for o in output:
        line = tokenizer.decode(o.to(device), skip_special_tokens=True,clean_up_tokenization_spaces=True) # tokenizer.decode(): 토큰 인덱스를 실제 텍스트로 디코딩
        #print(line)
        result.append(line)
    return result

In [ ]:
pairs.head()

,Unnamed: 0,texts,pairs
0,0,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,9,사람이 고양이를 천장에 던지고 있다.,사람이 고양이를 천장에 던진다.
2,11,한 여성이 아기를 안아서 캥거루를 안는다.,한 여성이 아기를 안아서 팔에 캥거루를 안는다.
3,13,사람이 종이 한 장을 접고 있다.,누군가가 종이를 접고 있다.
4,16,북극곰이 눈 위에서 미끄러지고 있다.,북극곰이 눈 위로 미끄러져 가고 있다.


In [ ]:
for i in pairs[500:520].iterrows():
    text = i[1]["pairs"]
    print(f"{text} => {getParaphrased(text)[3]}")

망명을 위해 스노우든의 안타 => 스노우던의 망명은 안타리에게 결정을 내린다.
노벨 도리스 레딩 94세 사망 => 노벨 도리스 레딩 94세에 죽다 
보스턴 경찰은 마라톤 폭탄테러로 3명이 구금되었다고 밝혔다. => 3명의 부상자가 보스턴 경찰에 의해 체포되었다.
뉴욕디의 트위터 캠페인 역효과 => 뉴욕디는 트위터 광고에 대해 좋은 반응을 보이지 않는다.
터키 검색은 마지막으로 사라진 광부들이 발견한 것으로 끝납니다 => 터키는 결국 실종된 광부들에 의해 이루어졌으나 그들은 실종된 상태입니다.
앙키트 차반은 결혼하기 위해 보석금을 내주었다. => 결혼을 약속한 앙키트 차반은 귀한 선물로 현금으로 주었다.
베이루트 : 레바논 총리는 어제 취임한 지 10개월이 넘은 내각을 구성했는데, 이들 중 심각한 분열을 대부분 넘어선 후 광범위한 정치 단체를 포함했다 => 베이루트 대통령 (사진)은 어제 10개월 이상의 임기를 가진 레바논 총리 중 심각한 분지를 모두 삭제한 뒤 범야권의 정치 집단 진출과 안보를 위한 폭넓은 지역외교를 진행하는 일을 하고 있다.
세계 최고령의 남자가 116세에 사망 => 116년 전 세계 최고령 남성의 생이 사망했다.
그리스 극우 지도자 투옥 재판 중 => 그리스 극우파 지도자 체포 재판 중
넬슨 만델라는 95세에 사망한다. => 95세에 그는 세상을 하직하고 만다.
유대인 관례에 묻힌 나치 족장 => 나치 국적자인 호이자 씨는 유태인 관습 속에 새겨
스위스는 세계 최고 최저임금에 투표용지를 던졌다. => 세계에서 제일 낮은 최저 임금에도 스위스는 투표용지를 던졌다.
러드는 호주의 새 총리로 취임했다 => 새 총리에 러시가는 호주의 총리에 임명되었다.
강력한 삼촌을 해고한 배후에 있는 북한의 김모씨 => 김정은 체제 하에 있는 김동원(북한 출신) 총잡이 김모씨가 북한에서 김정일 위원장의 아들인 김모씨를 해고했다.
전 영국 PM 마가렛 대처가 87세에 사망 => 영국 PM 소속 인물인 영국의 쇼핑몰 쇼핑몰 직원인 영국의 마가렛 대처는 올해 87세가

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/T5model_225376data")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/T5model_225376data")

In [ ]:
def getParaphrased(s):
    encoded = tokenizer(s, padding=True, truncation = True, return_tensors = 'pt')
    output = model.generate(
        input_ids=encoded["input_ids"].to(device), attention_mask=encoded["attention_mask"].to(device), # input_ids = 인코딩된 입력 문장의 인덱스 텐서 / attention_mask = 인코딩된 입력 문장의 어텐션 마스크 텐서(0=패딩토큰, 1=실제토큰)
        max_length=len(s)*2, # 생성되는 문장의 최대 출력 길이
        do_sample=True, # 샘플링 여부 파라미터(True=다양한 토큰을 샘플링하여 출력 생성 / False=모델이 가장 확률적으로 높은 토큰을 선택하여 출력 생성)
        top_k=30, # 상위 k개의 확률 분포에서만 토큰을 샘플링하는데 사용되는 파라미터. 값을 높이면 생성 다양성 증가. 낮추면 생성 정확성 증가.
        top_p=0.5, # 누적 확률 분포의 상위 p%에 해당하는 토큰만을 고려하여 샘플링하는데 사용되는 파라미터. 값을 높이면 생성 정확성이 증가, 낮추면 생성 다양성이 증가.
        early_stopping=False, # 조기 중단 여부 결정 파라미터(True=모델이 '<eos>' 토큰을 만났을 때 생성 중단 / False=max_length까지 계속해서 토큰 생성)
        num_return_sequences=5 # 생성할 문장 개수 지정 파라미터. 출력 시퀀스 1개로 설정
    )
    result = []
    for o in output:
        line = tokenizer.decode(o.to(device), skip_special_tokens=True,clean_up_tokenization_spaces=True) # tokenizer.decode(): 토큰 인덱스를 실제 텍스트로 디코딩
        #print(line)
        result.append(line)
    return result

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(50358, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):